### LESSON 1. 
# PYTORCH. ACTIVATION FUNCTIONS. FULLY CONNECTED

____

In [2]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 8.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.1 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 2.2 MB/s eta 0:00:0000:0100:010m
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/00/45/ec3407adf6f6b5bf867a4462b2b0af27597a26bd3cd6e2534cb6ab029938/filelock-3.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions from https://files.pythonhosted.org/packages/ec/6b/63cc3df74987c36fe26157ee12e09e8f9db4de771e0f3404263117e75b95/typing_extensions-4.7.1-py3-none-any.whl.metadata
  Using cached typing_extensions-4.7.1-py3-none-any.whl.metadata (3.1 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

In [57]:
import torch

### Task 5

In [21]:
def function01(tensor: torch.Tensor, count_over: str) -> torch.Tensor:
    if count_over == 'columns':
        # mean value of columns
        return tensor.mean(dim=0)
    if count_over == 'rows':
        # mean value of rows
        return tensor.mean(dim=1)
    else:
        raise ValueError(f"Invalid value for count_over: {count_over}. It should be 'columns' or 'rows'.")

In [51]:
random_tensor = torch.rand(2, 3)

In [52]:
random_tensor

tensor([[0.9539, 0.4972, 0.0732],
        [0.0954, 0.3607, 0.2042]])

In [53]:
function01(random_tensor, 'rows')

tensor([0.5081, 0.2201])

### Task 6

In [54]:
def function02(dataset: torch.Tensor):
    columns = dataset.shape[1]
    return torch.rand(columns, requires_grad=True, dtype=torch.float32)

In [55]:
random_tensor

tensor([[0.9539, 0.4972, 0.0732],
        [0.0954, 0.3607, 0.2042]])

In [56]:
function02(random_tensor)

tensor([0.7044, 0.6944, 0.5493], requires_grad=True)

### Task 7

In [93]:
def function03(x: torch.Tensor, y: torch.Tensor):
    # Fix random seed
    torch.manual_seed(0)

    # Fix n_step, step_size and maximum value of mse
    step_size = 1e-2
    max_mse = 1
    max_iterations = 5000

    # Initial mse value (to enter the loop)
    mse = float('inf')
    iteration = 0
    # Determine n_features, n_objects, weights and X
    n_features = x.shape[1]
    n_objects = y.shape[0]
    w = torch.rand(n_features, requires_grad=True, dtype=torch.float32)

    # Loop for determine mse by gradient
    while mse > 1:
        y_pred = torch.matmul(x, w) + torch.randn(n_objects) / 2
        mse = torch.mean((y_pred - y) ** 2)

        print(f'MSE {mse.item():.5f}')
        iteration += 1

        mse.backward()

        with torch.no_grad():
            w -= w.grad * step_size
            w.grad.zero_()

        if iteration > max_iterations:
            break

    return w


In [94]:
x = torch.rand(2, 3)

In [95]:
x

tensor([[0.8964, 0.4556, 0.6323],
        [0.3489, 0.4017, 0.0223]])

In [96]:
y = torch.rand(2)

In [97]:
y

tensor([0.1689, 0.2939])

In [98]:
function03(x, y)

MSE 0.40641


tensor([0.4876, 0.7633, 0.0830], requires_grad=True)

### Task 8

In [106]:
import torch
from torch import nn


def function04(x: torch.Tensor, y: torch.Tensor):
    # Fix random seed
    torch.manual_seed(0)

    # Determine n_features, n_objects, weights
    n_features = x.shape[1]

    # Create a fully connected layer
    layer = nn.Linear(in_features=n_features, out_features=1, bias=False)

    # Ensure the layer uses the desired initial weights
    with torch.no_grad():
        layer.weight.copy_(torch.rand(n_features, requires_grad=True, dtype=torch.float32))

    # Define step size, max MSE, and max iterations
    step_size = 1e-2
    max_mse = 0.3
    max_iterations = 5000

    # Initial mse value (to enter the loop)
    mse = float('inf')
    iteration = 0

    # Loop for determine mse by gradient
    while mse >= max_mse:
        y_pred = layer(x).squeeze()  # Use the layer for predictions
        mse = torch.mean((y_pred - y) ** 2)

        print(f'Iteration {iteration}, MSE {mse.item():.5f}')
        mse.backward()

        with torch.no_grad():
            layer.weight.data -= step_size * layer.weight.grad
            layer.zero_grad()


        iteration += 1

        if iteration > max_iterations:
            print("Reached the maximum number of iterations without achieving the desired MSE!")
            break

    return layer

In [107]:
x = torch.rand(2, 3)
y = torch.rand(2)

In [108]:
function04(x, y)

Iteration 0, MSE 0.22240


Linear(in_features=3, out_features=1, bias=False)

___